# 論文資料及整理
* YELP
    * EDA
    * 50 review above 
    * join business, check-in and tips

# Import

In [2]:
import pandas as pd
import json
import numpy as np
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt

In [3]:
business_path = '/home/adam/Steph_C/my_thesis/Data/yelp/yelp_academic_dataset_business.json'
checkin_path = '/home/adam/Steph_C/my_thesis/Data/yelp/yelp_academic_dataset_checkin.json'
tip_path = '/home/adam/Steph_C/my_thesis/Data/yelp/yelp_academic_dataset_tip.json'
review_path = '/home/adam/Steph_C/my_thesis/Data/yelp/yelp_academic_dataset_review.json'

# pickle file
business_pk = '/home/adam/Steph_C/my_thesis/Data/yelp/business.pkl'
checkin_pk = '/home/adam/Steph_C/my_thesis/Data/yelp/checkin.pkl'
tip_pk = '/home/adam/Steph_C/my_thesis/Data/yelp/tip.pkl'
review_pk = '/home/adam/Steph_C/my_thesis/Data/yelp/review.pkl'
final_pk = '/home/adam/Steph_C/my_thesis/Data/yelp/final.pkl'

# Load Data

In [ ]:
# unhash this part if necessary

# load review data
# r_dict = []

# r_dtypes = {"stars": np.float16, 
#             "useful": np.int32, 
#             "funny": np.int32,
#             "cool": np.int32,
#            }
# with open(review_path, "r") as f:
#     reader = pd.read_json(f, orient="records", lines=True, 
#                           dtype=r_dtypes, chunksize=1000)
#     for chunk in reader:
#         # get 5 years data from '2018-2022'
#         reduced_chunk = chunk.drop(columns=['review_id', 'user_id'])\
#                              .query("date >= '2017-1-01'").query("date <= '2022-12-31'")
#         r_dict.append(reduced_chunk)
    
# r_dict = pd.concat(r_dict, ignore_index=True)

# # load business data
# b_dict =[]

# with open(business_path, "r") as f:
#     reader = pd.read_json(f, orient="records", lines=True, chunksize=1000)
        
#     for chunk in reader:
#         b_dict.append(chunk)
    
# b_dict = pd.concat(b_dict, ignore_index=True)

# # load check-in data
# c_dict =[]

# with open(checkin_path, "r") as f:
#     reader = pd.read_json(f, orient="records", lines=True, chunksize=1000)
        
#     for chunk in reader:
# #         reduced_chunk = chunk.drop(
# #             columns=['postal_code','latitude','longitude','is_open','attributes','hours'])
#         c_dict.append(chunk)
    
# c_dict = pd.concat(c_dict, ignore_index=True)

# # load tip data
# t_dict =[]

# with open(tip_path, "r") as f:
#     reader = pd.read_json(f, orient="records", lines=True, chunksize=1000)
        
#     for chunk in reader:
#         reduced_chunk = chunk.drop(
#             columns=['user_id'])
#         t_dict.append(reduced_chunk)
    
# t_dict = pd.concat(t_dict, ignore_index=True)

In [ ]:
# business_df = pd.DataFrame(b_dict)
# review_df = pd.DataFrame(r_dict)
# checkin_df = pd.DataFrame(c_dict)
# tip_df = pd.DataFrame(t_dict)

In [ ]:
# review_df.to_pickle("../Data/yelp/review.pkl")  
# business_df.to_pickle("../Data/yelp/business.pkl") 
# checkin_df.to_pickle("../Data/yelp/checkin.pkl")  
# tip_df.to_pickle("../Data/yelp/tip.pkl")  

# Join tables

In [3]:
business_df = pd.read_pickle (business_pk)
review_df = pd.read_pickle(review_pk)
# checkin_df = pd.read_pickle(checkin_pk)
# tip_df = pd.read_pickle(tip_pk)

# print shape
# print(review_df.shape)
# print(business_df.shape)
# print(checkin_df.shape)
# print(tip_df.shape)


In [4]:
#### All sorts of processing 

# get 50 reviews above 
review_df = review_df[review_df.groupby('business_id')['business_id'].transform('count')>=20]
print(review_df.shape)

# get only the open stores
business_df = business_df[business_df.is_open==1]
print(business_df.shape)

# # spread the check-in data
# checkin_df = checkin_df.assign(date=checkin_df.date.str.split(','))
# checkin_df = checkin_df.explode('date')
# print(checkin_df.shape)

(3117235, 7)
(119698, 14)


In [5]:
# join tables
final_df = review_df.merge(business_df, on='business_id')

In [6]:
final_df.shape

(2812563, 20)

In [7]:
# create only restaurant dataset
drop_l = []
for i in range(len(final_df)):
    try:
        if 'Restaurant' not in final_df.categories[i]:
            drop_l.append(final_df.business_id[i])
    except:
        pass
drop_l = list(set(drop_l))

df = final_df[~final_df.business_id.isin(drop_l)]

print(df.shape)



# create with 
venues_df = final_df[final_df.business_id.isin(drop_l)]
print(venues_df.shape)

(2112553, 20)
(700010, 20)


In [8]:
df.to_pickle("../Data/yelp/restaurant_only.pkl")  
venues_df.to_pickle("../Data/yelp/other_venues.pkl")  

# Trim Dataset

In [15]:
df = pd.read_pickle('../Data/yelp/restaurant_only.pkl')

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
# # get restaurants with 2-12 postal codes
# final_df = pd.DataFrame()
# for i in Counter(df.name):
#     tmp = df[df.name==i].reset_index(drop=True)
#     drop_l = [i for i in range(len(tmp)) if not tmp.postal_code[i].isdigit()]
#     tmp = tmp.drop(drop_l)
#     if len(Counter(tmp.postal_code))>1 and len(Counter(tmp.postal_code)) <=12:
#         final_df = pd.concat([final_df, tmp])

In [19]:
# # get restaurants with 2-12 postal codes

# 排除掉偏鄉地帶
postal_res_count = {}
for i in Counter(df.postal_code):
    tmp = df[df.postal_code==i]
    postal_res_count[i]=len(Counter(tmp.name))

drop_l = [i for i in postal_res_count if postal_res_count[i]<5]
df = df[~df.postal_code.isin(drop_l)]


final_df = pd.DataFrame()
for i in Counter(df.name):
    tmp = df[df.name==i].reset_index(drop=True)
    drop_l = [i for i in range(len(tmp)) if not tmp.postal_code[i].isdigit()]
    tmp = tmp.drop(drop_l)
    if len(Counter(tmp.postal_code))>2 and len(Counter(tmp.postal_code)) <=12:
        final_df = pd.concat([final_df, tmp])

In [20]:
final_df.shape

(192547, 20)

In [21]:
# it should be 565
len(Counter(final_df.name))

405

In [22]:
final_df.to_pickle("../data/ORI_by_postal_code_s_dropped.pkl")

In [ ]:
# ../data/ORI_by_postal_code_s_dropped.pkl for 3 above branches